In [8]:
import pandas as pd 

def calculate_score(company_value, industry_avg_value, weight, score_type):
    if pd.isna(company_value):
        return 0

    if score_type == 'disclosure':
        return 100
    if score_type == 'lower_is_better':
        diff = company_value - industry_avg_value
        if diff < -0.5 * industry_avg_value:
            return 100
        elif diff < -0.2 * industry_avg_value:
            return 75
        elif abs(diff) <= 0.2 * industry_avg_value:
            return 50
        elif diff < 0.5 * industry_avg_value:
            return 25
        else:
            return 0

    if score_type == 'higher_is_better':
        diff = company_value - industry_avg_value
        if diff < -0.5 * industry_avg_value:
            return 0
        elif diff < -0.2 * industry_avg_value:
            return 25
        elif abs(diff) <= 0.2 * industry_avg_value:
            return 50
        elif diff < 0.5 * industry_avg_value:
            return 75
        else:
            return 100

    if score_type == 'closest_to_average':
        diff = company_value - industry_avg_value
        if abs(diff) <= 0.2 * industry_avg_value:
            return 100
        elif diff < -0.5 * industry_avg_value or diff > 0.5 * industry_avg_value:
            return 0
        else:
            return 50

    if score_type == 'turnover_level':
        if 10 <= company_value <= 20:
            return 100
        elif 0 <= company_value < 10 or 20 < company_value <= 30:
            return 50
        else:
            return 0
    elif score_type == 'adverse_events':
        return 100 if company_value == 0 else 0
    elif score_type == 'Work-related_injuries':
        if company_value == 0:
            return 100
        elif 1 <= company_value <= 2:
            return 50
        else:
            return 0

# Function to calculate the rating based on the total score
def calculate_rating(score):
    if 85.72 <= score <= 100:
        return 'AAA'
    elif 71.44 <= score < 85.72:
        return 'AA'
    elif 57.15 <= score < 71.44:
        return 'A'
    elif 42.87 <= score < 57.15:
        return 'BBB'
    elif 28.58 <= score < 42.86:
        return 'BB'
    elif 14.30 <= score < 28.57:
        return 'B'
    elif 0.0 <= score < 14.29:
        return 'CCC'
    else:
        return 'Unknown'

def calculate_esg_score(company_esg_data, industry_avg_data, company_name='AEM'):
    # 过滤出指定公司（例如 AEM）的数据
    company_esg_data = company_esg_data[company_esg_data['company'] == company_name]

    # 将行业数据转换为字典以便快速查找
    industry_avg_dict = industry_avg_data.set_index(['categories', 'issue', 'metric'])['value'].to_dict()

    total_score = 0
    e_score = 0
    s_score = 0
    g_score = 0
    total_weight = 0

    # 初始化26个指定指标的默认值为0
    metrics_values = {
        "Emission intensities": 0,
        "Energy consumption intensity": 0,
        "Waste generated": 0,
        "Water intensity": 0,
        "Board independence": 0,
        "Women in the management team": 0,
        "Women on the board": 0,
        "Percentage of employees covered by health insurance": 0,
        "Company donated": 0,
        "Average training hours per employee": 0,
        "Current employees by age groups": 0,
        "Current employees by gender": 0,
        "Employee satisfaction rate": 0,
        "New hires by gender": 0,
        "New hires by age": 0,
        "Total turnover": 0,
        "Turnover by gender": 0,
        "Turnover by age": 0,
        "Fatalities": 0,
        "High-consequence injuries": 0,
        "Work-related injuries": 0,
    }

    for idx, row in company_esg_data.iterrows():
        category = row['categories']
        issue = row['issue']
        metric = row['metric']
        company_value = row['value']
        weight = row['weight']

        industry_avg_value = industry_avg_dict.get((category, issue, metric), None)
        if industry_avg_value is None:
            continue

        # 根据指标确定得分类型
        score_type = ""
        if metric in ["Protected or restored habitats", "Absolute emissions", "Total energy consumption", 
                      "Green financing projects", "Green certified buildings", "water consumption", 
                      "External audit conducted", "Anti-corruption disclosures", "Anti-corruption training", 
                      "Assurance of sustainability report", "List of relevant certifications", 
                      "Availability of Healthcare Resources", "Community Health Program", 
                      "Philanthropic initiatives", "Controversial Sourcing", "Consumer rights protection", 
                      "Total number of employees"] :
            score_type = "disclosure"
        elif metric in ["Emission intensities", "Energy consumption intensity", "Waste generated", "Water intensity"]:
            score_type = "lower_is_better"
        elif metric in ["Board independence", "Women in the management team", "Women on the board", 
                        "Percentage of employees covered by health insurance", "Company donated", 
                        "Average training hours per employee", "Employee satisfaction rate"]:
            score_type = "higher_is_better"
        elif metric in ["Total turnover"]:
            score_type = "turnover_level"
        elif metric in ["Fatalities", "High-consequence injuries"]:
            score_type = "adverse_events"
        elif metric in ["Work-related injuries"]:
            score_type = "Work-related_injuries"
        else:
            score_type = "closest_to_average"

        # 计算当前指标得分
        score = calculate_score(company_value, industry_avg_value, weight, score_type)
        total_score += score * weight
        total_weight += weight
        if category == 'Environment':
            e_score += score * weight
        elif category == 'Social':
            s_score += score * weight
        elif category == 'Governance':
            g_score += score * weight

        # 更新26个指定指标的得分
        if metric in metrics_values:
            metrics_values[metric] = score

    # 计算最终分数
    total_score = (total_score / total_weight) if total_weight > 0 else 0
    e_score = (e_score / total_weight) if total_weight > 0 else 0
    s_score = (s_score / total_weight) if total_weight > 0 else 0
    g_score = (g_score / total_weight) if total_weight > 0 else 0

    # 计算评级
    rating = calculate_rating(total_score)

    # 构建包含26个指标的输出表格
    result = {
        'Company': company_name, 
        'Total Score': total_score,
        'Rating': rating,
        'E Score': e_score,
        'S Score': s_score,
        'G Score': g_score
    }
    result.update(metrics_values)

    return result

# 加载数据
company_esg_data = pd.read_excel('D:/Desktop/dss5105/project/company_esg_data.xlsx')
industry_avg_data = pd.read_excel('D:/Desktop/dss5105/project/industry_avg_data.xlsx')

# 计算ESG得分 (选择OUEH公司)
company_score_data = calculate_esg_score(company_esg_data, industry_avg_data, company_name='AEM')

# 转换为DataFrame显示结果并保存为Excel
result_df = pd.DataFrame([company_score_data])
print(result_df)


  Company  Total Score Rating    E Score    S Score    G Score  \
0     AEM     71.55424     AA  29.880775  29.601927  12.071539   

   Emission intensities  Energy consumption intensity  Waste generated  \
0                   100                           100               75   

   Water intensity  ...  Current employees by gender  \
0                0  ...                          100   

   Employee satisfaction rate  New hires by gender  New hires by age  \
0                           0                  100                 0   

   Total turnover  Turnover by gender  Turnover by age  Fatalities  \
0              50                  50               50         100   

   High-consequence injuries  Work-related injuries  
0                        100                    100  

[1 rows x 27 columns]


In [9]:
result_df

,Company,Total Score,Rating,E Score,S Score,G Score,Emission intensities,Energy consumption intensity,Waste generated,Water intensity,...,Current employees by gender,Employee satisfaction rate,New hires by gender,New hires by age,Total turnover,Turnover by gender,Turnover by age,Fatalities,High-consequence injuries,Work-related injuries
0,AEM,71.55424,AA,29.880775,29.601927,12.071539,100,100,75,0,...,100,0,100,0,50,50,50,100,100,100


In [12]:
import pandas as pd

def calculate_score(company_value, industry_avg_value, weight, score_type):
    # Same as before...
    if pd.isna(company_value):
        return 0

    if score_type == 'disclosure':
        return 100
    if score_type == 'lower_is_better':
        diff = company_value - industry_avg_value
        if diff < -0.5 * industry_avg_value:
            return 100
        elif diff < -0.2 * industry_avg_value:
            return 75
        elif abs(diff) <= 0.2 * industry_avg_value:
            return 50
        elif diff < 0.5 * industry_avg_value:
            return 25
        else:
            return 0

    if score_type == 'higher_is_better':
        diff = company_value - industry_avg_value
        if diff < -0.5 * industry_avg_value:
            return 0
        elif diff < -0.2 * industry_avg_value:
            return 25
        elif abs(diff) <= 0.2 * industry_avg_value:
            return 50
        elif diff < 0.5 * industry_avg_value:
            return 75
        else:
            return 100

    if score_type == 'closest_to_average':
        diff = company_value - industry_avg_value
        if abs(diff) <= 0.2 * industry_avg_value:
            return 100
        elif diff < -0.5 * industry_avg_value or diff > 0.5 * industry_avg_value:
            return 0
        else:
            return 50

    if score_type == 'turnover_level':
        if 10 <= company_value <= 20:
            return 100
        elif 0 <= company_value < 10 or 20 < company_value <= 30:
            return 50
        else:
            return 0
    elif score_type == 'adverse_events':
        return 100 if company_value == 0 else 0
    elif score_type == 'Work-related_injuries':
        if company_value == 0:
            return 100
        elif 1 <= company_value <= 2:
            return 50
        else:
            return 0

def calculate_rating(score):
    if 85.72 <= score <= 100:
        return 'AAA'
    elif 71.44 <= score < 85.72:
        return 'AA'
    elif 57.15 <= score < 71.44:
        return 'A'
    elif 42.87 <= score < 57.15:
        return 'BBB'
    elif 28.58 <= score < 42.86:
        return 'BB'
    elif 14.30 <= score < 28.57:
        return 'B'
    elif 0.0 <= score < 14.29:
        return 'CCC'
    else:
        return 'Unknown'

def calculate_esg_score(company_esg_data, industry_avg_data, company_name='AEM'):
    company_esg_data = company_esg_data[company_esg_data['company'] == company_name]
    industry_avg_dict = industry_avg_data.set_index(['categories', 'issue', 'metric'])['value'].to_dict()

    total_score = 0
    e_score = 0
    s_score = 0
    g_score = 0
    total_weight = 0

    metrics_values = {
        "Emission intensities": 0,
        "Energy consumption intensity": 0,
        "Waste generated": 0,
        "Water intensity": 0,
        "Board independence": 0,
        "Women in the management team": 0,
        "Women on the board": 0,
        "Percentage of employees covered by health insurance": 0,
        "Company donated": 0,
        "Average training hours per employee": 0,
        "Current employees by age groups": 0,
        "Current employees by gender": 0,
        "Employee satisfaction rate": 0,
        "New hires by gender": 0,
        "New hires by age": 0,
        "Total turnover": 0,
        "Turnover by gender": 0,
        "Turnover by age": 0,
        "Fatalities": 0,
        "High-consequence injuries": 0,
        "Work-related injuries": 0,
    }

    for idx, row in company_esg_data.iterrows():
        category = row['categories']
        issue = row['issue']
        metric = row['metric']
        company_value = row['value']
        weight = row['weight']

        industry_avg_value = industry_avg_dict.get((category, issue, metric), None)
        if industry_avg_value is None:
            continue

        score_type = ""
        if metric in ["Protected or restored habitats", "Absolute emissions", "Total energy consumption", 
                      "Green financing projects", "Green certified buildings", "water consumption", 
                      "External audit conducted", "Anti-corruption disclosures", "Anti-corruption training", 
                      "Assurance of sustainability report", "List of relevant certifications", 
                      "Availability of Healthcare Resources", "Community Health Program", 
                      "Philanthropic initiatives", "Controversial Sourcing", "Consumer rights protection", 
                      "Total number of employees"]:
            score_type = "disclosure"
        elif metric in ["Emission intensities", "Energy consumption intensity", "Waste generated", "Water intensity"]:
            score_type = "lower_is_better"
        elif metric in ["Board independence", "Women in the management team", "Women on the board", 
                        "Percentage of employees covered by health insurance", "Company donated", 
                        "Average training hours per employee", "Employee satisfaction rate"]:
            score_type = "higher_is_better"
        elif metric in ["Total turnover"]:
            score_type = "turnover_level"
        elif metric in ["Fatalities", "High-consequence injuries"]:
            score_type = "adverse_events"
        elif metric in ["Work-related injuries"]:
            score_type = "Work-related_injuries"
        else:
            score_type = "closest_to_average"

        score = calculate_score(company_value, industry_avg_value, weight, score_type)
        total_score += score * weight
        total_weight += weight
        if category == 'Environment':
            e_score += score * weight
        elif category == 'Social':
            s_score += score * weight
        elif category == 'Governance':
            g_score += score * weight

        if metric in metrics_values:
            metrics_values[metric] = score

    total_score = (total_score / total_weight) if total_weight > 0 else 0
    e_score = (e_score / total_weight) if total_weight > 0 else 0
    s_score = (s_score / total_weight) if total_weight > 0 else 0
    g_score = (g_score / total_weight) if total_weight > 0 else 0

    rating = calculate_rating(total_score)

    result = {
        'Company': company_name, 
        'Total Score': total_score,
        'Rating': rating,
        'E Score': e_score,
        'S Score': s_score,
        'G Score': g_score
    }
    result.update(metrics_values)

    return result

# Load data
company_esg_data = pd.read_excel('D:/Desktop/dss5105/project/company_esg_data.xlsx')
industry_avg_data = pd.read_excel('D:/Desktop/dss5105/project/industry_avg_data.xlsx')

# Loop over all unique company names in the dataset and calculate ESG scores
all_companies = company_esg_data['company'].unique()
all_results = []

for company_name in all_companies:
    company_score_data = calculate_esg_score(company_esg_data, industry_avg_data, company_name)
    all_results.append(company_score_data)

# Convert results to DataFrame and save to Excel
result_df = pd.DataFrame(all_results)
result_df.to_excel('D:/Desktop/dss5105/project/all_companies_esg_scores.xlsx', index=False)

# Optionally, print the results
print(result_df)


            Company  Total Score Rating    E Score    S Score    G Score  \
0               AEM    71.554240     AA  29.880775  29.601927  12.071539   
1               AML    57.991799      A  21.087928  24.776443  12.127429   
2           Medtecs    42.064093     BB  23.602835   5.141579  13.319679   
3              OUEH    69.167413      A  27.160952  26.134622  15.871838   
4               RMG    65.937328      A  24.217588  26.909752  14.809989   
5               TGL    53.921665    BBB  16.840528  31.045367   6.035769   
6            Abbott    49.981380    BBB  16.542468  17.101343  16.337568   
7   Setsco Services    38.162214     BB  12.853951  15.583958   9.724304   
8           Haw Par    28.763047     BB   5.812219   8.569299  14.381529   
9       iXBiopharma    55.588677    BBB  25.167660  18.330848  12.090169   
10         TopGlove    49.552910    BBB  25.260802  15.350223   8.941884   
11             UGHC    43.368106    BBB  21.236958   8.159459  13.971689   

    Emissio

In [13]:
result_df

,Company,Total Score,Rating,E Score,S Score,G Score,Emission intensities,Energy consumption intensity,Waste generated,Water intensity,...,Current employees by gender,Employee satisfaction rate,New hires by gender,New hires by age,Total turnover,Turnover by gender,Turnover by age,Fatalities,High-consequence injuries,Work-related injuries
0,AEM,71.554240,AA,29.880775,29.601927,12.071539,100,100,75,0,...,100,0,100,0,50,50,50,100,100,100
1,AML,57.991799,A,21.087928,24.776443,12.127429,100,100,0,0,...,50,0,50,50,100,50,50,100,100,100
2,Medtecs,42.064093,BB,23.602835,5.141579,13.319679,25,50,100,0,...,0,0,0,0,0,0,0,0,0,0
3,OUEH,69.167413,A,27.160952,26.134622,15.871838,50,100,0,0,...,100,0,0,0,50,100,0,100,100,100
4,RMG,65.937328,A,24.217588,26.909752,14.809989,0,100,0,0,...,50,0,50,0,50,50,50,100,100,100
5,TGL,53.921665,BBB,16.840528,31.045367,6.035769,100,0,0,0,...,50,0,100,50,100,50,50,100,100,100
6,Abbott,49.981380,BBB,16.542468,17.101343,16.337568,100,100,0,0,...,0,0,0,0,100,0,0,100,0,0
7,Setsco Services,38.162214,BB,12.853951,15.583958,9.724304,50,100,75,0,...,0,0,0,0,0,0,50,100,100,50
8,Haw Par,28.763047,BB,5.812219,8.569299,14.381529,0,0,0,0,...,100,0,0,0,0,0,0,100,0,0
9,iXBiopharma,55.588677,BBB,25.167660,18.330848,12.090169,25,100,100,0,...,100,0,0,0,0,0,0,100,100,100
